In [84]:
%pip install scipy statsmodels tqdm
import numpy as np
import scipy.stats as stats
from tqdm import tqdm as tqdm
import pandas as pd
import random
import pandas as pd
from scipy.optimize import bisect
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind
from statsmodels.stats.multitest import multipletests


[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [85]:
df = pd.read_csv(
    filepath_or_buffer="data1.csv"
)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1601993 entries, 0 to 1601992
Data columns (total 2 columns):
 #   Column   Non-Null Count    Dtype 
---  ------   --------------    ----- 
 0   user_id  1601993 non-null  object
 1   gmv      1601993 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 24.4+ MB


In [86]:

groupped = df.groupby(by="user_id").agg("sum").reset_index()
df

,user_id,gmv
0,guo0pdwqvg,506
1,guo0pdwqvg,760
2,guo0pdwqvg,633
3,guo0pdwqvg,506
4,guo0pdwqvg,760
...,...,...
1601988,3vbm7rz5vu,910
1601989,3vbm7rz5vu,809
1601990,3vbm7rz5vu,1011
1601991,npbrhympo0,1349


In [87]:
mat_long = groupped["gmv"].mean()

In [88]:
var_long = groupped["gmv"].var(ddof=1)

mat_r = 2750.847
var_r = 3646228.409
alpha = 0.05
beta = 0.2

In [89]:
def size_mde(var, effect, k = 1, alpha = 0.05, beta = 0.2):
    dist = stats.norm(loc=0, scale=1)
    const = (dist.ppf(1- alpha/2) + dist.ppf(1- beta))**2 
    return const*(1+k)*var/(effect**2)

In [90]:
effect = mat_r*0.01
k = 3
m = size_mde(effect=effect, var = var_r, k = k, alpha=alpha, beta=beta)
n = m / k
total_size = m*(k+1)/k
total_size

np.float64(201705.34238964328)

In [91]:

mat_r = 2750.847
var_r = 3646228.409
alpha = 0.05
beta = 0.2
k = 3  # соотношение контрольной к тестовой группе (3:1)

# Точный расчет размера выборки
def calculate_sample_size(mean, variance, mde_percent, k=1, alpha=0.05, beta=0.2):
    # Преобразуем MDE из процентов в абсолютное значение
    effect_size = mean * (mde_percent / 100)
    
    # Квантили нормального распределения
    z_alpha = stats.norm.ppf(1 - alpha/2)
    z_beta = stats.norm.ppf(1 - beta)
    
    # Расчет размера тестовой группы (n2)
    n2 = (variance * (1 + 1/k) * (z_alpha + z_beta)**2) / (effect_size**2)
    
    # Размер контрольной группы (n1 = k * n2)
    n1 = k * n2
    
    # Общий размер выборки
    total_size = n1 + n2
    
    return np.ceil(n2), np.ceil(n1), np.ceil(total_size)

# Расчет для MDE = 1%
test_size, control_size, total_size = calculate_sample_size(
    mean=mat_r,
    variance=var_r,
    mde_percent=1,
    k=k,
    alpha=alpha,
    beta=beta
)

total_size

np.float64(201706.0)